In [ ]:
#%%capture
#!pip install --upgrade mediapipe

In [ ]:
# @title Импорт библиотек
import os
import cv2
import mediapipe as mp
import numpy as np
import math
from math import acos, degrees
import time
import pandas as pd

In [ ]:
# @title Список точек для определения углов
poins_for_angles = []
# Стопы
poins_for_angles.append((32, 28, 26)) # ang_0_(28)
poins_for_angles.append((31, 27, 25)) # ang_1_(27)
# Колени
poins_for_angles.append((28, 26, 24)) # ang_2_(26)
poins_for_angles.append((27, 25, 23)) # ang_3_(25)
# Пах
poins_for_angles.append((26, 24, 23)) # ang_4_(24)
poins_for_angles.append((25, 23, 24)) # ang_5_(23)
# Изгиб тела
poins_for_angles.append((26, 24, 12)) # ang_6_(24)
poins_for_angles.append((25, 23, 11)) # ang_7_(23)
# Плечи
poins_for_angles.append((24, 12, 14)) # ang_8_(12)
poins_for_angles.append((23, 11, 13)) # ang_9_(11)
# Локти
poins_for_angles.append((12, 14, 16)) # ang_10_(14)
poins_for_angles.append((11, 13, 15)) # ang_11_(13)
# Глаза
poins_for_angles.append((11, 12, 5))  # ang_12_(12)
poins_for_angles.append((12, 11, 2))  # ang_13_(11)
# Большие пальцы
poins_for_angles.append((14, 16, 22)) # ang_14_(14_16_22)
poins_for_angles.append((13, 15, 21)) # ang_15_(13_15_21)
# Колено-плечо-локоть
poins_for_angles.append((26, 12, 14)) # ang_16_(26_12_14)
poins_for_angles.append((25, 11, 13)) # ang_17_(25_11_13)
# Колено-плечо-кисть
poins_for_angles.append((26, 12, 16)) # ang_18_(26_12_16)
poins_for_angles.append((25, 11, 15)) # ang_19_(25_11_15)
# Пятки
poins_for_angles.append((28, 30, 32)) # ang_20_(28_30_32)
poins_for_angles.append((27, 29, 31)) # ang_21_(27_29_31)
# Пятки с колен
poins_for_angles.append((26, 30, 32)) # ang_22_(26_30_32)
poins_for_angles.append((25, 29, 31)) # ang_23_(25_29_31)

landmark_poins_for_angles = ["ang_0_(28)", "ang_1_(27)", "ang_2_(26)", "ang_3_(25)",
                             "ang_4_(24)", "ang_5_(23)", "ang_6_(24)", "ang_7_(23)",
                             "ang_8_(12)", "ang_9_(11)", "ang_10_(14)", "ang_11_(13)",
                             "ang_12_(12)", "ang_13_(11)", "ang_14_(14_16_22)", "ang_15_(13_15_21)",
                             "ang_16_(26_12_14)", "ang_17_(25_11_13)", "ang_18_(26_12_16)", "ang_19_(25_11_15)",
                             "ang_20_(28_30_32)", "ang_21_(27_29_31)", "ang_22_(26_30_32)", "ang_23_(25_29_31)"]

In [ ]:
# @title Функция определения угла по 3 точкам

def calculate_triangle_angles(point1, point2, point3):
    # Вычисление длин сторон треугольника
    side_a = math.dist(point1, point2)
    side_b = math.dist(point2, point3)
    side_c = math.dist(point3, point1)

    # Вычисление углов треугольника с помощью формулы косинусов
    #angle_A = math.degrees(math.acos((side_c**2 + side_a**2 - side_b**2) / (2 * side_c * side_a)))
    angle_B = math.degrees(math.acos((side_a**2 + side_b**2 - side_c**2) / (2 * side_a * side_b)))
    #angle_C = math.degrees(math.acos((side_b**2 + side_c**2 - side_a**2) / (2 * side_b * side_c)))

    return int(round(angle_B))
    #return angle_B

def angle_on_3_points_3d(point_a, point_b, point_c):
  v1 = point_b - point_a  # Определяем вектор v1
  v2 = point_c - point_b  # Определяем вектор v2
  # Скалярное произведение векторов – это произведение их модулей (длин) на косинус между ними...
  # ... соответственно cos = скалярное произведение (np.dot) делённое на произведение модулей (длин) векторов (np.linalg.norm)
  cos = np.dot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
  rad = acos(cos)
  #ang = degrees(rad)

  if cos < 0:
    ang = degrees(rad)  # Угол в градусах
  else:
    ang = 180 -degrees(rad)  # Угол в градусах

  return int(round(ang))
  #return ang

def get_angles(pose_landmarks):
    angles = []
    for points in poins_for_angles:
        i, j, k = points
        point_a = np.array([pose_landmarks[i].x, pose_landmarks[i].y, pose_landmarks[i].z])
        point_b = np.array([pose_landmarks[j].x, pose_landmarks[j].y, pose_landmarks[j].z])
        point_c = np.array([pose_landmarks[k].x, pose_landmarks[k].y, pose_landmarks[k].z])
        #angles.append(calculate_triangle_angles(point_a, point_b, point_c))
        angles.append(angle_on_3_points_3d(point_a, point_b, point_c))
    return angles

def results_to_angles(capture_file, results, frame_num, aug_num):
    pose_landmarks = results.pose_landmarks.landmark
    capture_file = capture_file.split('.')[0] # Отбрасываем расширение
    row = capture_file.split('_')
    row[0] = row[0].split('-')[0] # Оставляем только номер удара без названия
    row.append(aug_num) # Добавляем Номер аугментации
    row.append(frame_num) # Добавляем номер кадра
    row.extend(get_angles(pose_landmarks)) # Добавляем углы
    return np.array(row)


In [ ]:
# @title 1. ФУНКЦИЯ ИЗМЕНЕНИЯ МАСШТАБА ВИДЕО (Copy)
def resize_frame(frame, scale_percent):
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
    return resized_frame

# @title 2. ФУНКЦИЯ ПОВОРОТА ВИДЕО НА УГОЛ (Copy)
def rotate_frame(frame, angle):
    height, width = frame.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_frame = cv2.warpAffine(frame, rotation_matrix, (width, height))
    return rotated_frame

In [ ]:
# @title Функция распознавания видео

def video_to_ang(capture_file):

  mp_drawing = mp.solutions.drawing_utils
  mp_pose = mp.solutions.pose
  pose_list0 = [] # 0 - без аугментации
  pose_list1 = [] # 1 - изменение масштаба
  pose_list2 = [] # 2 - поворот
  pose_list3 = [] # 3 - размытие
  pose_list4 = [] # 4 - приближение
  pose_list5 = [] # 5 - медианное размытие

  # Создаём объект VideoCapture
  cap = cv2.VideoCapture(capture_file)

  # Setup mediapipe instance
  with mp_pose.Pose(static_image_mode=True, model_complexity=2) as pose:
      while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Перевод кадра в RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Делаем распазнование 0 - без аугментации
        results = pose.process(image)
        if not results.pose_landmarks:
            continue
        angles = results_to_angles(capture_file, results, len(pose_list0), 0)
        pose_list0.append(angles)

        # Делаем распознавание 1 - изменение масштаба (33%)
        resize_image = resize_frame(image, 33)
        results = pose.process(resize_image)
        if not results.pose_landmarks:
            continue
        angles = results_to_angles(capture_file, results, len(pose_list1), 1)
        pose_list1.append(angles)

        # Делаем распознавание 2 - поворот на 15 градусов
        results = pose.process(rotate_frame(image, 15))
        if not results.pose_landmarks:
            continue
        angles = results_to_angles(capture_file, results, len(pose_list2), 2)
        pose_list2.append(angles)

        # Делаем распознавание 3 - размытие изображения ядром (10,10)
        blur_image = image
        cv2.blur(blur_image, (10,10))
        results = pose.process(blur_image)
        if not results.pose_landmarks:
            continue
        angles = results_to_angles(capture_file, results, len(pose_list3), 3)
        pose_list3.append(angles)

        # Делаем распознавание 4 - приближение (обрезаем 1/3 по ширине и высоте)
        crop_image = cv2.cvtColor(frame[180:1080, 320:1600], cv2.COLOR_BGR2RGB)
        results = pose.process(crop_image)
        if not results.pose_landmarks:
            continue
        angles = results_to_angles(capture_file, results, len(pose_list4), 4)
        pose_list4.append(angles)

        #crop_image = cv2.cvtColor(crop_image, cv2.COLOR_BGR2RGB)
        #mp_drawing.draw_landmarks(crop_image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        #                          mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
        #                          mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        #                         )
        #pose_landmarks = results.pose_landmarks.landmark
        #for i in range(len(poins_for_angles)):
        #    j= poins_for_angles[i][1]
        #    point = (int(pose_landmarks[j].x * crop_image.shape[1]), int(pose_landmarks[j].y * crop_image.shape[0]))
        #    cv2.putText(crop_image, str(pose_list4[-1][i+7]), point, cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 1, cv2.LINE_AA)
        #cv2.imwrite("C:\\Karate\\test4" + str(len(pose_list4)) + ".png", crop_image)

        # Делаем распознавание 5 - медианное размытие с ядром 5
        median_blur_image = image
        cv2.medianBlur(median_blur_image, 5)
        results = pose.process(median_blur_image)
        if not results.pose_landmarks:
            continue
        angles = results_to_angles(capture_file, results, len(pose_list5), 5)
        pose_list5.append(angles)

      cap.release()

  return np.array(pose_list0 + pose_list1 + pose_list2 + pose_list3 + pose_list4 + pose_list5)

In [ ]:
def collect_files(folder_path):
    files_list = []
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            files_list.append(os.path.join(dirpath, filename))
    return files_list

In [ ]:
base_dir = "C:\Karate\Videos"
os.chdir(base_dir)
folders_name_list = []
for folder_name in sorted(os.listdir()):
    folders_name_list.append(folder_name)

In [ ]:
os.environ['TZ'] = 'Europe/Moscow'

columns = ["class", "view", "belt", "karateka", "num", "aug", "frame"]
columns += landmark_poins_for_angles

start_time = time.localtime()
print("Старт обработки файлов в:", time.strftime("%H:%M:%S", time.localtime()))

for i in range(len(folders_name_list)):
    files_list = collect_files(folders_name_list[i])
    df = pd.DataFrame(columns = columns)
    out_csv_file = "C:\\Karate\\" + str(i) + ".csv"
    df.to_csv(out_csv_file, index=False)

    for file in files_list[:]:
        pose_list = video_to_ang(file)
        df = pd.DataFrame(pose_list)
        df.to_csv(out_csv_file, header=False, index=False, mode='a')
        print("Файл", file, "обработан в:", time.strftime("%H:%M:%S", time.localtime()))
    print("Каталог", folders_name_list[i], "обработан в:", time.strftime("%H:%M:%S", time.localtime()))

print("Начало обработки в:", time.strftime("%H:%M:%S", start_time), "окончание в:", time.strftime("%H:%M:%S", time.localtime()))

Старт обработки файлов в: 22:15:13
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_bb_01_01.mp4 обработан в: 22:16:05
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_bb_01_02.mp4 обработан в: 22:16:38
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_bb_01_03.mp4 обработан в: 22:17:07
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_bb_01_04.mp4 обработан в: 22:17:37
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_bb_01_05.mp4 обработан в: 22:18:05
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_bb_01_06.mp4 обработан в: 22:18:33
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_bb_01_07.mp4 обработан в: 22:19:05
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_bb_01_08.mp4 обработан в: 22:19:35
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_bb_01_09.mp4 обработан в: 22:20:07
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_bb_01_10.mp4 обработан в: 22:20:38
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_gb_01_01.mp4 обработан в: 22:21:20
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_f_gb_01_02.mp4 обр

Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_rb_01_01.mp4 обработан в: 06:44:16
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_rb_01_02.mp4 обработан в: 06:44:50
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_rb_01_03.mp4 обработан в: 06:45:25
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_rb_01_04.mp4 обработан в: 06:45:57
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_rb_01_05.mp4 обработан в: 06:46:29
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_rb_01_06.mp4 обработан в: 06:47:04
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_rb_01_07.mp4 обработан в: 06:47:44
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_rb_01_08.mp4 обработан в: 06:48:21
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_rb_01_09.mp4 обработан в: 06:49:03
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_rb_01_10.mp4 обработан в: 06:49:41
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_wb_01_01.mp4 обработан в: 06:50:09
Файл 01-ГЕДАН БАРАЙ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\01_p_wb_01_02.mp4 обработан в: 06:50:37
Файл 01-ГЕДАН БА

Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_01_06.mp4 обработан в: 07:28:39
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_01_07.mp4 обработан в: 07:29:03
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_01_08.mp4 обработан в: 07:29:29
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_01_09.mp4 обработан в: 07:29:54
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_01_10.mp4 обработан в: 07:30:15
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_02_01.mp4 обработан в: 07:30:39
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_02_02.mp4 обработан в: 07:31:04
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_02_03.mp4 обработан в: 07:31:25
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_02_04.mp4 обработан в: 07:31:45
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_02_05.mp4 обработан в: 07:32:07
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУКОЙ)\02_p_gb_02_06.mp4 обработан в: 07:32:29
Файл 02-ГЬЯКУ ДЗУКИ (ПРЯМОЙ УДАР ЗАДНЕЙ РУК

Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_rb_01_09.mp4 обработан в: 09:09:46
Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_rb_01_10.mp4 обработан в: 09:10:08
Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_wb_01_01.mp4 обработан в: 09:10:33
Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_wb_01_02.mp4 обработан в: 09:11:01
Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_wb_01_03.mp4 обработан в: 09:11:28
Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_wb_01_04.mp4 обработан в: 09:11:54
Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_wb_01_05.mp4 обработан в: 09:12:18
Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_wb_01_06.mp4 обработан в: 09:12:46
Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_wb_01_07.mp4 обработан в: 09:13:11
Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_wb_01_08.mp4 обработан в: 09:13:38
Файл 03-КИДЗАМИ ДЗУКИ (ПРЯМОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\03_f_wb_01_09.mp4 обработан в: 09:14:06

Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_01_09.mp4 обработан в: 09:48:01
Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_01_10.mp4 обработан в: 09:48:30
Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_02_01.mp4 обработан в: 09:48:54
Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_02_02.mp4 обработан в: 09:49:22
Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_02_03.mp4 обработан в: 09:49:47
Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_02_04.mp4 обработан в: 09:50:13
Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_02_05.mp4 обработан в: 09:50:37
Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_02_06.mp4 обработан в: 09:51:04
Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_02_07.mp4 обработан в: 09:51:30
Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_02_08.mp4 обработан в: 09:51:55
Файл 04-МАВАШИ  ДЗУКИ (КРУГОВОЙ УДАР ЗАДНЕЙ РУКОЙ)\04_f_gb_02_09.mp4 обработан в: 09:52:21

...

Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_gb_02_06.mp4 обработан в: 13:06:29
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_gb_02_07.mp4 обработан в: 13:07:04
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_gb_02_08.mp4 обработан в: 13:07:40
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_gb_02_09.mp4 обработан в: 13:08:15
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_gb_02_10.mp4 обработан в: 13:08:47
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_pb_01_01.mp4 обработан в: 13:09:07
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_pb_01_02.mp4 обработан в: 13:09:21
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_pb_01_03.mp4 обработан в: 13:09:48
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_pb_01_04.mp4 обработан в: 13:10:24
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_pb_01_05.mp4 обработан в: 13:10:53
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_pb_01_06.mp4 обработан в: 13:11:22
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_pb_01_07.mp4 обработан в: 13:11:52
Файл 07-СОТО УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\07_f_pb_01_08

Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_f_bb_01_08.mp4 обработан в: 14:03:45
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_f_bb_01_09.mp4 обработан в: 14:04:33
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_f_bb_01_10.mp4 обработан в: 14:05:13
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_f_gb_01_01.mp4 обработан в: 14:05:54
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_f_gb_01_02.mp4 обработан в: 14:06:32
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_f_gb_01_03.mp4 обработан в: 14:07:11
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_f_gb_01_04.mp4 обработан в: 14:07:51
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_f_gb_01_05.mp4 обработан в: 14:08:34
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_f_gb_01_06.mp4 обработан в: 14:09:17
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_f_gb_01

Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_p_gb_02_08.mp4 обработан в: 14:58:15
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_p_gb_02_09.mp4 обработан в: 14:58:52
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_p_gb_02_10.mp4 обработан в: 14:59:29
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_p_pb_01_01.mp4 обработан в: 15:00:08
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_p_pb_01_02.mp4 обработан в: 15:00:45
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_p_pb_01_03.mp4 обработан в: 15:01:19
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_p_pb_01_04.mp4 обработан в: 15:01:52
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_p_pb_01_05.mp4 обработан в: 15:02:27
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_p_pb_01_06.mp4 обработан в: 15:03:01
Файл 08-УРА МАВАШИ ГЕРИ (ОБРАТНЫЙ КРУГОВОЙ  УДАР ЗАДНЕЙ НОГОЙ)\08_p_pb_01

Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_f_wb_01_04.mp4 обработан в: 15:41:33
Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_f_wb_01_05.mp4 обработан в: 15:42:01
Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_f_wb_01_06.mp4 обработан в: 15:42:31
Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_f_wb_01_07.mp4 обработан в: 15:43:02
Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_f_wb_01_08.mp4 обработан в: 15:43:30
Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_f_wb_01_09.mp4 обработан в: 15:44:00
Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_f_wb_01_10.mp4 обработан в: 15:44:30
Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_p_bb_01_01.mp4 обработан в: 15:45:03
Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_p_bb_01_02.mp4 обработан в: 15:45:30
Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_p_bb_01_03.mp4 обработан в: 15:45:56
Файл 09-УРАКЕН УЧИ (КРУГОВОЙ УДАР ПЕРЕДНЕЙ РУКОЙ)\09_p_bb_01_04.mp4 обработан в: 15:46:23
Файл 09-УР

Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_gb_02_08.mp4 обработан в: 16:25:40
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_gb_02_09.mp4 обработан в: 16:26:12
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_gb_02_10.mp4 обработан в: 16:26:42
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_pb_01_01.mp4 обработан в: 16:27:11
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_pb_01_02.mp4 обработан в: 16:27:37
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_pb_01_03.mp4 обработан в: 16:28:03
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_pb_01_04.mp4 обработан в: 16:28:31
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_pb_01_05.mp4 обработан в: 16:29:00
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_pb_01_06.mp4 обработан в: 16:29:33
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_pb_01_07.mp4 обработан в: 16:30:00
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_pb_01_08.mp4 обработан в: 16:30:33
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_pb_01_09.mp4 обработан в: 16:30:58
Файл 10-УЧИ УКЕ (БЛОК ПЕРЕДНЕЙ РУКОЙ)\10_f_pb_01_10.mp4 обработа

In [ ]:
df = pd.read_csv("C:\\Karate\\0.csv")
#df
df[df["frame"] == 0]

,class,view,belt,karateka,num,aug,frame,ang_0_(28),ang_1_(27),ang_2_(26),...,ang_14_(14_16_22),ang_15_(13_15_21),ang_16_(26_12_14),ang_17_(25_11_13),ang_18_(26_12_16),ang_19_(25_11_15),ang_20_(28_30_32),ang_21_(27_29_31),ang_22_(26_30_32),ang_23_(25_29_31)
0,1,f,bb,1,1,0,0,114,117,152,...,133,118,135,158,103,125,122,122,129,112
1,1,f,bb,1,1,0,1,114,123,159,...,132,116,128,161,97,127,125,127,130,118
2,1,f,bb,1,1,0,2,116,122,156,...,135,116,128,159,97,126,123,125,131,116
3,1,f,bb,1,1,0,3,115,118,154,...,129,111,129,160,98,128,124,125,132,114
4,1,f,bb,1,1,0,4,109,122,161,...,125,115,127,161,97,128,117,126,126,116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38941,1,p,wb,1,10,5,48,97,102,141,...,120,142,138,152,107,98,94,117,90,120
38942,1,p,wb,1,10,5,49,99,104,131,...,118,139,130,144,100,101,96,116,107,120
38943,1,p,wb,1,10,5,50,99,104,127,...,119,142,130,150,100,104,95,115,107,122
38944,1,p,wb,1,10,5,51,94,102,130,...,121,148,130,150,99,95,92,116,102,120
